# Supervised Learning with AMEX Data
## American Express - Default Prediction

In [ ]:
import pandas as pd

#Cargar el dataset
df = pd.read_csv('./test_data.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

#Visualizar la composición del target
plt.figure(figsize=(5,5))
ax = sn.countplot(x="target", data=df)

In [ ]:
# Separar el target de las features
y = df.target
X = df.drop(['target'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir la data en train y test
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,
                                                                random_state=0)

In [ ]:
# "Cardinalidad" significa el número de valores únicos en una columna
# Selecciono columnas categóricas con cardinalidad relativamente baja (conveniente pero arbitrario)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Selecciono columnas numéricas
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Conservo solo las columnas seleccionadas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

# Construimos el pipeline completo en tres pasos.

* Paso 1: definimos los pasos de preprocesamiento
De manera similar a cómo un pipeline agrupa los pasos de preprocesamiento y modelado, usamos la clase ColumnTransformer para agrupar diferentes pasos de preprocesamiento.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

* Paso 2 definimos el modelo

In [ ]:
from sklearn.linear_model import LogisticRegression

# Inicializamos el modelo
model = LogisticRegression()


* Paso 3: creamos y evaluamos el pipeline

Finalmente, usamos la clase Pipeline para definir un pipeline que agrupa los pasos de preprocesamiento y modelado.

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

# Paquete de preprocesamiento y código de modelado en una canalización
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocesamiento de datos de entrenamiento, modelo de ajuste
my_pipeline.fit(X_train, y_train)

# Preprocesamiento de datos de validación, obtener predicciones
preds = my_pipeline.predict(X_valid)

# Evaluación del modelo
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)
print(classification_report(y_valid, preds))

Ademas chequeamos matriz de confución

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_valid,preds)
print(cm)
accuracy_score(y_valid, preds)